In [4]:
from tqdm.notebook import tqdm
from time import sleep

In [6]:
l = [8, 9, 8]
m = [2, 3, 4]
l = [(i, e) for i, e in tqdm(enumerate(l))]

0it [00:00, ?it/s]

In [2]:
from tqdm import trange, tqdm
from time import sleep

for i in trange(3, desc='1st loop'):
    for j in tqdm(range(100), desc='2nd loop'):
        sleep(0.01)

1st loop: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.05s/it]
